## Meu classificador de frutas, legumes e verduras

Fruta: É um tipo de alimento derivado de plantas que geralmente é caracterizado por seu sabor doce, aroma distinto e elevado teor de suco. As frutas podem incluir frutos verdadeiros, como maçãs e bananas, bem como pseudofrutos, como morangos e tomates.<br /><br />

Legume: É uma categoria ampla de alimentos derivados de plantas que inclui frutos, talos, raízes e sementes. Os legumes são frequentemente consumidos como parte de uma dieta saudável e são ricos em vitaminas, minerais e outros nutrientes importantes.<br /><br />

Verdura: São vegetais que podem ser consumidos como alimento, incluindo folhas, flores, botões ou talos da planta. As verduras são uma fonte importante de vitaminas, minerais e fibras, e são frequentemente incluídas em dietas saudáveis para complementar a ingestão de outros nutrientes.<br /><br />

Frutas, legumes e verduras são tipos diferentes de alimentos vegetais que pertencem a diferentes categorias. Frutas são produtos da flor da planta, geralmente caracterizados por seu sabor adocicado e aroma distinto. Legumes incluem frutos, raízes, caules e sementes da planta que são utilizados na alimentação. Verduras são partes comestíveis da planta, como folhas, flores, botões ou hastes. Em resumo, frutas são as sementes envoltas em polpa adocicada, legumes são os vegetais comestíveis que não são frutos e verduras são os vegetais folhosos.

    db_frutas: https://drive.google.com/file/d/1XkoW4CyR99KH2mmur0ZCICXYSBOTO8cD/view?usp=share_link
    db_legumes: https://drive.google.com/file/d/1XvC8nu2GPfEHOMSKWqAHgOn2mGApBrUF/view?usp=share_link
    db_verduras: https://drive.google.com/file/d/1Xvh_mgyLS-t437XuUha_vMVTx8nqdWjX/view?usp=share_link

In [19]:
#@title download libs
!pip install pandas
!pip install scipy
!pip install sklearn
!pip install pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [20]:
import requests
import re
import pandas as pd
import numpy as np

In [21]:
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
def upload_databases(name, url):
    response = requests.get(url)
    print(f">> db: {name}, response: {response};")
    text = response.content.decode("utf-8")
    return text

In [24]:
url_db_frutas = "https://docs.google.com/uc?export=download&id=1XkoW4CyR99KH2mmur0ZCICXYSBOTO8cD"
url_db_legumes = "https://docs.google.com/uc?export=download&id=1XvC8nu2GPfEHOMSKWqAHgOn2mGApBrUF"
url_db_verduras = "https://docs.google.com/uc?export=download&id=1Xvh_mgyLS-t437XuUha_vMVTx8nqdWjX"

db_frutas = upload_databases("db_frutas", url_db_frutas)
db_legumes = upload_databases("db_legumes", url_db_legumes)
db_verduras = upload_databases("db_verduras", url_db_verduras)

>> db: db_frutas, response: <Response [200]>;
>> db: db_legumes, response: <Response [200]>;
>> db: db_verduras, response: <Response [200]>;


In [25]:
print("db_frutas: '{}...',\ndb_legumes: '{}...',\ndb_verduras: '{}...'".format(
    db_frutas[:55], db_legumes[:55], db_verduras[:55])
)


db_frutas: 'Abacate; Abacaxi; Abiu; Araçá; Azeitona; Açaí; Acerola;...',
db_legumes: 'Abóbora; Abobrinha; Aipim; Alho; Amaranto; Amêndoa; Ame...',
db_verduras: 'Abóbora (flor); Açafrão; Açafrão; Acelga; Acelga-chines...'


In [26]:
#@title processamento

def processador_de_texto(data):
    data = re.sub('[^\w\s]','', str(data.lower()))
    data = data.replace("\r", " ")
    data = data.replace("\n", " ")
    data = data.replace("  ", " ")
    data = data.split(" ")
    return data

In [27]:
db_list = [
    ["Fruta", db_frutas], ["Legume", db_legumes], ["Verdura", db_verduras],
]

In [28]:
db_processed = [[db_list[i][0], i, processador_de_texto(db_list[i][1])] for i, _ in enumerate(db_list)]
print(db_processed)

[['Fruta', 0, ['abacate', 'abacaxi', 'abiu', 'araçá', 'azeitona', 'açaí', 'acerola', 'ameixa', 'amora', 'abricó', 'anonácea', 'atemoya', 'abóbora', 'babaco', 'bacaba', 'bacuribanana', 'baru', 'buriti', 'bilimbi', 'biribá', 'butiá', 'cabeludinha', 'cacau', 'cagaita', 'cajámanga', 'cereja', 'caimito', 'cajá', 'conde', 'cupuaçu', 'caju', 'calabaça', 'carambola', 'calabura', 'coco', 'calamondin', 'cambucá', 'cagaita', 'cambuci', 'caqui', 'damasco', 'dovyalis', 'durião', 'embaúba', 'feijoa', 'figo', 'framboesa', 'frutapão', 'frutas', 'do', 'cerrado', 'frutadoconde', 'glicosmis', 'goiaba', 'granadilla', 'graviola', 'groselha', 'grumixama', 'guabiju', 'guabiroba', 'guaraná', 'guariroba', 'heisteria', 'himbeere', 'ilama', 'ingá', 'jabuticaba', 'jaca', 'jambo', 'jambolão', 'jaracatiá', 'jamelão', 'jatobá', 'jenipapo', 'jerivá', 'jujuba', 'kiwi', 'langsat', 'laranja', 'lichia', 'limão', 'limas', 'ácidas', 'e', 'doces', 'longan', 'lucuma', 'mabolo', 'maçã', 'macadâmia', 'mamão', 'mamey', 'mamonci

In [29]:
print("> frutas:", len(db_processed[0][2]))
print("> legumes:", len(db_processed[1][2]))
print("> verduras:", len(db_processed[2][2]))

> frutas: 167
> legumes: 133
> verduras: 136


In [30]:
dataset_horticolas = pd.DataFrame(columns=["horticolas", "classe", "labels"])
for name, num, lista in db_processed:

    for i in lista:
        dataset_horticolas = dataset_horticolas.append(
            {'horticolas': i, 'classe': name, 'labels': float(num)}, ignore_index=True
        )

print(dataset_horticolas.shape)
display(dataset_horticolas.head())
print(dataset_horticolas["labels"].value_counts())

(436, 3)


,horticolas,classe,labels
0,abacate,Fruta,0.0
1,abacaxi,Fruta,0.0
2,abiu,Fruta,0.0
3,araçá,Fruta,0.0
4,azeitona,Fruta,0.0


0.0    167
2.0    136
1.0    133
Name: labels, dtype: int64


In [31]:
#@title Tokenizer

def save_vector(vector):
    # Abre um arquivo em modo de gravação binária
    with open(f"hortalicia_vectorizer.pickle", "wb") as f:
        # Salva o objeto TfidfVectorizer no arquivo usando o pickle
        pickle.dump(vector, f)
    print(">> 'hortalicia_vectorizer.pickle' has been created")
    return

vectorizer = TfidfVectorizer()
myvectorizer = vectorizer.fit(dataset_horticolas["horticolas"])
save_vector(myvectorizer)

>> 'hortalicia_vectorizer.pickle' has been created


In [32]:
X = myvectorizer.fit_transform(dataset_horticolas["horticolas"])
y = dataset_horticolas["labels"]

In [33]:
#@title Model

def generate_machine_learning_m(X, y):
    
    model = RandomForestClassifier()

    model.fit(X, y)

    result_y = model.score(X, y); print(f">> model precicion: {result_y:.2%}")
    return model

def downlaod_model(model):
    with open("hortalicia_model.pkl", "wb") as f:
        pickle.dump(model, f)
    print(">> 'hortalicia_model' has been downloaded!")
    return

In [34]:
model = generate_machine_learning_m(X, y)
downlaod_model(model)

>> model precicion: 91.28%
>> 'hortalicia_model' has been downloaded!


In [35]:
def classify_new(input_, myvectorizer):

    input_vect = myvectorizer.transform(input_)

    input_vect_array = np.array(input_vect)
    input_vect_array_csr_matrix = csr_matrix((input_vect_array).all())

    return input_vect_array_csr_matrix

In [36]:
#@title Classificar hortícola

classify = input("Digite um hortícola: ")

classify_vector = classify_new(processador_de_texto(classify), myvectorizer)

proba = model.predict_proba(classify_vector)
classification = model.predict(classify_vector)

src_class = dataset_horticolas.loc[ dataset_horticolas["labels"] == classification[0]]

print(f">> probabilidades: '{proba[0]}'")

print(">> '{}' ({}) seria a classificação correta para o hortícola '{}'!".format(
    src_class["classe"].values[0], classification[0], classify)
)

Digite um hortícola: maçã
>> probabilidades: '[1. 0. 0.]'
>> 'Fruta' (0.0) seria a classificação correta para o hortícola 'maçã'!
